# 🥈 Level 2: Analytics Apprentice

Welcome to Level 2 of the Data Science Sandbox! You've mastered the basics, now it's time to become an analytics apprentice. This level focuses on:

- **Advanced Data Cleaning**: Handle messy, real-world data with confidence
- **Statistical Analysis**: Master descriptive statistics, hypothesis testing, and correlation analysis
- **Data Validation**: Use modern tools like Pandera for automated data quality checks
- **Modern Data Science**: Work with cutting-edge tools and best practices

## 🎯 Learning Objectives

By the end of this level, you'll be able to:
- Clean and validate complex datasets using multiple strategies
- Perform comprehensive statistical analysis with proper interpretation
- Implement automated data quality checks and monitoring
- Use modern data science tools for professional workflows

## 📊 Datasets Used

- **Messy Sales Data**: E-commerce data with intentional quality issues
- **Analytics Data**: Customer behavior data for statistical analysis  
- **Survey Data**: Customer satisfaction and segmentation data

Let's begin your journey to becoming an analytics apprentice! 🚀

## 📚 Import Required Libraries

First, let's import all the libraries we'll need for our analytics journey:

In [ ]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np

# Statistical analysis
from scipy import stats
from scipy.stats import pearsonr, spearmanr, chi2_contingency, ttest_ind, f_oneway

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data validation and quality
import pandera as pa
from pandera import Column, DataFrameSchema, Check

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("🎯 Ready to start Level 2: Analytics Apprentice")

## 🧹 Challenge 1: Advanced Data Cleaning

Let's start with our first challenge - cleaning messy, real-world data. This dataset contains intentional data quality issues that you'll commonly encounter in practice.

### What You'll Learn:
- Multiple strategies for handling missing values
- Outlier detection using statistical methods (IQR, Z-score)
- Data validation using business rules
- Creating comprehensive data quality reports

In [ ]:
# Load the messy sales dataset
try:
    messy_sales = pd.read_csv('../data/datasets/messy_sales_data.csv')
    print(f"✅ Dataset loaded: {len(messy_sales)} records")
except FileNotFoundError:
    print("❌ Dataset not found. Let's create sample messy data:")
    # Create sample messy dataset
    np.random.seed(42)
    n_samples = 1000
    
    messy_sales = pd.DataFrame({
        'customer_id': range(1, n_samples + 1),
        'customer_name': ['Customer ' + str(i) if i % 10 != 0 else np.nan for i in range(1, n_samples + 1)],
        'customer_age': np.random.normal(35, 12, n_samples),
        'email': ['customer' + str(i) + '@email.com' if i % 15 != 0 else 'invalid_email' for i in range(1, n_samples + 1)],
        'phone': [f'+1-555-{str(i).zfill(4)}' if i % 20 != 0 else np.nan for i in range(1, n_samples + 1)],
        'sales': np.random.gamma(2, 100, n_samples),
        'product_category': np.random.choice(['Electronics', 'Clothing', 'Books', 'Home', 'Sports'], n_samples),
        'customer_satisfaction': np.random.choice([1, 2, 3, 4, 5, np.nan], n_samples, p=[0.05, 0.1, 0.2, 0.4, 0.2, 0.05]),
        'registration_date': pd.date_range('2020-01-01', periods=n_samples, freq='D'),
        'is_premium': np.random.choice([True, False], n_samples, p=[0.3, 0.7])
    })
    
    # Add intentional issues
    messy_sales.loc[messy_sales.index[:10], 'sales'] = messy_sales['sales'].max() * 10  # Extreme outliers
    messy_sales.loc[messy_sales.index[10:20], 'customer_age'] = -5  # Invalid ages
    messy_sales.loc[messy_sales.index[20:25], 'sales'] = -100  # Negative sales

print("📊 Dataset Overview:")
print(f"Shape: {messy_sales.shape}")
print(f"Memory usage: {messy_sales.memory_usage(deep=True).sum() / 1024:.1f} KB")

print("\n🔍 Basic Information:")
messy_sales.info()

### Step 1: Missing Values Analysis

Let's analyze the missing values in our dataset and develop strategies to handle them:

In [ ]:
# Comprehensive missing values analysis
missing_summary = messy_sales.isnull().sum()
missing_percentage = (missing_summary / len(messy_sales)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_summary,
    'Missing_Percentage': missing_percentage,
    'Data_Type': messy_sales.dtypes
}).sort_values('Missing_Count', ascending=False)

print("📊 Missing Values Analysis:")
print(missing_df[missing_df['Missing_Count'] > 0])

# Visualize missing values pattern
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Missing values heatmap
sns.heatmap(messy_sales.isnull(), cbar=True, ax=ax1, cmap='viridis')
ax1.set_title('Missing Values Heatmap')
ax1.set_xlabel('Columns')
ax1.set_ylabel('Rows (sample)')

# Missing values bar chart
missing_counts = messy_sales.isnull().sum()[messy_sales.isnull().sum() > 0]
missing_counts.plot(kind='bar', ax=ax2)
ax2.set_title('Missing Values Count by Column')
ax2.set_ylabel('Number of Missing Values')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\n💡 Insights:")
print(f"• Total missing values: {missing_summary.sum()}")
print(f"• Columns with missing data: {(missing_summary > 0).sum()}")
print(f"• Most affected column: {missing_summary.idxmax()} ({missing_summary.max()} missing)")